https://www.tutorialspoint.com/python_blockchain/python_blockchain_quick_guide.htm

# Import Libraries

In [1]:
# import libraries
import hashlib
import random
import string
import json
import binascii
import numpy as np
import pandas as pd
import pylab as pl
import logging
import datetime
import collections

In [2]:
!pip install pycryptodome

In [3]:
# following imports are required by PKI
import Crypto
import Crypto.Random
from Crypto.Hash import SHA
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5

# Function / Class Definition

## Client

In [4]:
class Client:
  def __init__(self):
    random = Crypto.Random.new().read
    self._private_key = RSA.generate(1024, random)
    self._public_key = self._private_key.publickey()
    self._signer = PKCS1_v1_5.new(self._private_key)

  @property
  def identity(self):
    return binascii.hexlify(self._public_key.exportKey(format='DER')).decode('ascii')

## Transaction

In [5]:
class Transaction:
  def __init__(self, sender, recipient, value):
    self.sender = sender
    self.recipient = recipient
    self.value = value
    self.time = datetime.datetime.now()

  def to_dict(self):
    if self.sender == "Genesis":
      identity = "Genesis"
    else:
      identity = self.sender.identity

    return collections.OrderedDict({
      'sender': identity,
      'recipient': self.recipient,
      'value': self.value,
      'time' : self.time})

  def sign_transaction(self):
    private_key = self.sender._private_key
    signer = PKCS1_v1_5.new(private_key)
    h = SHA.new(str(self.to_dict()).encode('utf8'))
    return binascii.hexlify(signer.sign(h)).decode('ascii')

In [6]:
def display_transaction(transaction):
  dict = transaction.to_dict()
  print("sender: " + dict['sender'])
  print('-----')
  print("recipient: " + dict['recipient'])
  print('-----')
  print("value: " + str(dict['value']))
  print('-----')
  print("time: " + str(dict['time']))
  print('-----')

## Block

In [7]:
class Block:
  def __init__(self):
    self.verified_transactions = []
    self.previous_block_hash = ""
    self.Nonce = ""

In [8]:
def dump_blockchain(self):
  print("Number of blocks in the chain: " + str(len(self)))
  for x in range(len(self)):
    block_temp = self[x]
    print("block # " + str(x))
    for transaction in block_temp.verified_transactions:
      display_transaction(transaction)
      print('--------------')
    print('=====================================')

## Miner

In [9]:
def sha256(message):
  return hashlib.sha256(message.encode('ascii')).hexdigest()

In [10]:
def mine(message, difficulty=1):
  assert difficulty >= 1
  prefix = '1' * difficulty
  for i in range(1000):
    digest = sha256(str(hash(message)) + str(i))
    if digest.startswith(prefix):
      print ("after " + str(i) + " iterations found nonce: "+ digest)
      return digest

# Sample Run

In [11]:
sample_chain = []

In [12]:
clientA = Client()
clientB = Client()
clientC = Client()
clientD = Client()

In [13]:
t0 = Transaction("Genesis", clientA.identity, "GENESIS")

block0 = Block()
block0.previous_block_hash = None
block0.verified_transactions.append(t0)

digest = hash(block0)
last_block_hash = digest

sample_chain.append(block0)

In [14]:
transactions = []

t1 = Transaction(clientA, clientB.identity, "Hello world!")
t1.sign_transaction()
transactions.append(t1)

t2 = Transaction(clientB, clientC.identity, "Testing123")
t2.sign_transaction()
transactions.append(t2)

t3 = Transaction(clientC, clientD.identity, "QWERTY")
t3.sign_transaction()
transactions.append(t3)

In [15]:
last_transaction_index = 0

block = Block()
for i in range(3):
  temp_transaction = transactions[last_transaction_index]
  # validate transaction
  # if valid
  block.verified_transactions.append(temp_transaction)
  last_transaction_index += 1

block.previous_block_hash = last_block_hash
block.Nonce = mine(block, 2)
digest = hash(block)
sample_chain.append(block)
last_block_hash = digest

after 74 iterations found nonce: 11f1bd0d400e1f3e15c28678125505a10ac2a2887612199b4542e90d11279081


In [16]:
dump_blockchain(sample_chain)

Number of blocks in the chain: 2
block # 0
sender: Genesis
-----
recipient: 30819f300d06092a864886f70d010101050003818d0030818902818100ba2ca0d2e55da37535496ab3ba8050e5040385a4e773b6ec51f3f7ef507db39783a69e45c8a24ef2e612e4e5c992dfebf013490d79578cd4ec731e16cc60231501253c138a0a762c67c6dd5196e27dc70d18226b89d407cadd199e2362defc349ecdd95bc6132dd9ca022369eaea0fe3911a3727dea83c215804e51da65dae410203010001
-----
value: GENESIS
-----
time: 2024-02-15 09:44:41.419833
-----
--------------
block # 1
sender: 30819f300d06092a864886f70d010101050003818d0030818902818100ba2ca0d2e55da37535496ab3ba8050e5040385a4e773b6ec51f3f7ef507db39783a69e45c8a24ef2e612e4e5c992dfebf013490d79578cd4ec731e16cc60231501253c138a0a762c67c6dd5196e27dc70d18226b89d407cadd199e2362defc349ecdd95bc6132dd9ca022369eaea0fe3911a3727dea83c215804e51da65dae410203010001
-----
recipient: 30819f300d06092a864886f70d010101050003818d0030818902818100d9c28c710023dc06d0861764011b4b2e8dbff935ab165ccc392cf7ad88aa0a37782cd0c7efd94f20adf50190dedf6d994872